# XGBoost (eXtreme Gradient Boosting)

XGBoost stands for **eXtreme Gradient Boosting**.
* **Gradient Boosting:** It belongs to the family of ensemble learning methods, specifically using the Gradient Boosting framework.
* **Ensemble Learning:** The core idea is to combine multiple "weak learners" (typically decision trees) to create a single "strong learner." The model "learns" from its previous mistakes.



### Analogy: The Student Learning from Mistakes
Imagine a student preparing for an exam:
1.  **First Try (Tree 1):** The student takes a practice test and gets a score. They analyze their mistakes.
2.  **Second Try (Tree 2):** They focus *specifically* on the topics they got wrong in the first test.
3.  **Repeat (Subsequent Trees):** They keep taking new tests, each time paying extra attention to the errors made in the previous test.
4.  **Final Exam (Prediction):** For the final exam, the student's knowledge is the sum of all the incremental learning from each practice test.

---

## How it Works: Step-by-Step

### 1. The High-Level Idea: Sequential Ensemble
Unlike Random Forest which builds trees in parallel, XGBoost builds trees **sequentially**. Each new tree is trained to correct the residual errors made by the combination of all previous trees.

### 2. The Mathematical Formulation

#### a) The Model
The final prediction for a data point $i$ after $K$ trees is:

$$\hat{y}_i = \phi(x_i) = \sum_{k=1}^{K} f_k(x_i)$$

* $\hat{y}_i$: Final predicted value.
* $x_i$: Feature vector for data point $i$.
* $f_k$: The prediction from the $k$-th decision tree.

#### b) The Objective Function
This is the heart of XGBoost. The goal is to minimize:

$$Obj(\theta) = \sum_i L(y_i, \hat{y}_i) + \sum_k \Omega(f_k)$$

This function has two critical parts:
1.  **Training Loss ($L$):** Measures how well our model fits the training data (e.g., Squared Error $(y_i - \hat{y}_i)^2$).
2.  **Regularization Term ($\Omega$):** Penalizes model complexity to prevent overfitting.
    $$\Omega(f) = \gamma T + \frac{1}{2}\lambda ||w||^2$$
    * $T$: Number of leaves in the tree.
    * $w$: The score (prediction value) on each leaf.
    * $\gamma$ (gamma): Minimum loss reduction required to make a split.
    * $\lambda$ (lambda): L2 regularization term on leaf weights.

### 3. The Learning Process: Additive Training
We start with an initial prediction (e.g., mean value).

* **Step 0:** $\hat{y}_i^{(0)} = 0$
* **Step 1:** Build tree $f_1$ to minimize the objective. $\hat{y}_i^{(1)} = \hat{y}_i^{(0)} + f_1(x_i)$
* **Step t:** Add tree $f_t$ that improves the model the most.
    $$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)$$

---

## How is the Next Tree Built? (Gradient Boosting)

We use **Gradient Descent** in function space. We don't have parameters like weights in a neural network; our "parameters" are the trees themselves.

**1. Compute Gradients:**
For a given loss function (e.g., Squared Loss), we calculate the gradient ($g_i$) and Hessian ($h_i$) for each data point with respect to the previous prediction.
* $g_i = \partial L / \partial \hat{y} = -2(y_i - \hat{y}_i)$ (First derivative)
* $h_i = \partial^2 L / \partial \hat{y}^2 = 2$ (Second derivative)

**2. Find Optimal Leaf Weights:**
Once the tree structure is fixed, the optimal weight $w_j^*$ for leaf $j$ is calculated analytically:

$$w_j^* = - \frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} h_i + \lambda}$$

**3. Calculate the Objective Reduction (The "Gain"):**
The algorithm decides where to split the tree by maximizing this **Gain** formula:

$$Gain = \frac{1}{2} \left[ \frac{(\sum_{L} g_i)^2}{\sum_{L} h_i + \lambda} + \frac{(\sum_{R} g_i)^2}{\sum_{R} h_i + \lambda} - \frac{(\sum_{P} g_i)^2}{\sum_{P} h_i + \lambda} \right] - \gamma$$

> **Key Insight:** Notice the $-\gamma$ at the end. This means the split must improve the loss by at least $\gamma$ to be considered valid. This is XGBoost's built-in pruning mechanism.

---

### Summary: Why XGBoost Wins

| Feature | Meaning |
| :--- | :--- |
| **Regularization (L1 & L2)** | Avoids overfitting (unlike standard Gradient Boosting). |
| **Parallel Tree Building** | Uses all CPU cores during the split-finding phase. |
| **Missing Values** | Automatically learns the best direction for missing data. |
| **Tree Pruning** | Uses "max_depth" parameter and prunes backwards. |
| **Shrinkage (`eta`)** | Scales down the weights of new trees to prevent overfitting. |

### When to Use
* **Target ($y$):** Continuous (Regression) or Categories (Classification).
* **Features ($X$):** Numeric (mostly), but handles sparse data well.
* **Performance:** When accuracy is the #1 priority (Kaggle competitions).

# XGBoost: Regressor vs. Classifier

Just like Random Forest and Decision Trees, XGBoost uses the same underlying tree-boosting framework but adapts its **Objective Function**, **Loss Function**, and **Output** depending on the task.

### 1. Problem Type & Output

| Feature | XGBoost **Classifier** | XGBoost **Regressor** |
| :--- | :--- | :--- |
| **Problem Type** | Classification (Categories) | Regression (Continuous Numbers) |
| **Prediction Goal** | Class labels or Probabilities | Continuous quantities |
| **Example Output** | `0` or `1`, `[0.2, 0.8]` | `250000`, `125.7`, `-2.45` |
| **Python Class** | `XGBClassifier` | `XGBRegressor` |

### 2. Objectives & Loss Functions

| Feature | XGBoost **Classifier** | XGBoost **Regressor** |
| :--- | :--- | :--- |
| **Default Objective** | `binary:logistic` (Binary)<br>`multi:softmax` (Multiclass) | `reg:squarederror` (Default)<br>`reg:absoluteerror` |
| **Loss Function** | Log Loss (Binary Cross-Entropy) | MSE (Mean Squared Error), MAE |
| **Mathematical Goal** | Maximize Likelihood | Minimize Error Distance |

---

### 3. When to Use Which?

#### Use XGBoost **Classifier** when:
* Predicting categories (e.g., Spam/Ham, Fraud/Not Fraud, Customer Churn).
* You need probability outputs (e.g., `predict_proba()` to get "80% chance of rain").
* You are evaluating with classification metrics: **Accuracy, Precision, Recall, F1-Score, AUC-ROC**.

#### Use XGBoost **Regressor** when:
* Predicting continuous numerical values (e.g., House Prices, Stock Value, Temperature).
* Forecasting quantities (e.g., "How many units will we sell next month?").
* You are evaluating with regression metrics: **RMSE (Root Mean Squared Error), MAE, $R^2$ Score**.

---

### 4. Deep Dive: Key Technical Differences

| Component | Regressor | Classifier |
| :--- | :--- | :--- |
| **Target Variable ($y$)** | Numeric (Continuous) | Categorical (Discrete) |
| **Output Transformation** | Raw Number (Identity) | **Sigmoid** (Binary) or **Softmax** (Multiclass) |
| **Gradient Calculation** | Simple difference: $\text{pred} - \text{target}$ | Computed using derivative of Log Loss |
| **Hessian Calculation** | Constant ($2$ for MSE) | $p(1-p)$ (where $p$ is probability) |

> **Summary:**
> * **XGBoost Regressor** $\rightarrow$ Numeric prediction, Regression Loss.
> * **XGBoost Classifier** $\rightarrow$ Probability prediction, Classification Loss.
> * **The Core:** The tree-building and boosting framework is **identical**. To switch from Regressor to Classifier, XGBoost simply changes the **Loss Function**, the **Gradient/Hessian** formulas, and the final **Output Mapping**.

# The Math: How XGBoost Switches Modes

XGBoost uses a **Second-Order Taylor Expansion** to approximate the loss function. This means for *every* split decision, it needs two numbers for every data point:
1.  **Gradient ($g_i$):** The "slope" (First Derivative of Loss).
2.  **Hessian ($h_i$):** The "curvature" (Second Derivative of Loss).

$$Obj \approx \sum_{i=1}^{n} [L(y_i, \hat{y}_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2} h_i f_t^2(x_i)] + \Omega(f_t)$$

---

### 1. XGBoost Regressor (Linear Regression)
**Goal:** Minimize the distance between prediction and target.

* **Loss Function:** Mean Squared Error (MSE)
    $$L(\theta) = \frac{1}{2} (y_i - \hat{y}_i)^2$$
    *(Note: The 1/2 is added to make the derivative cleaner)*

* **Step 1: The Gradient ($g_i$)**
    Take the first derivative with respect to the prediction $\hat{y}$:
    $$g_i = \frac{\partial L}{\partial \hat{y}} = (\hat{y}_i - y_i)$$
    *Interpretation:* The error (Residual).

* **Step 2: The Hessian ($h_i$)**
    Take the derivative of the gradient (Second derivative):
    $$h_i = \frac{\partial^2 L}{\partial \hat{y}^2} = 1$$
    *Interpretation:* The curvature is constant for squared error.

* **Final Output:**
    The leaf weights are calculated directly using these values. The output is a raw number (e.g., $250,000$).

---

### 2. XGBoost Classifier (Binary Logistic)
**Goal:** Minimize the probability error (Log Loss).

* **Transformation:** The model outputs a "log-odds" score ($z$), which is converted to a probability ($p$) using the Sigmoid function:
    $$p = \sigma(z) = \frac{1}{1 + e^{-z}}$$

* **Loss Function:** Binary Cross-Entropy (Log Loss)
    $$L(\theta) = -[y_i \ln(p_i) + (1 - y_i) \ln(1 - p_i)]$$

* **Step 1: The Gradient ($g_i$)**
    This derivation is complex, but the result is elegant:
    $$g_i = \frac{\partial L}{\partial z} = p_i - y_i$$
    *Interpretation:* The difference between the predicted probability and the actual class (0 or 1).

* **Step 2: The Hessian ($h_i$)**
    The second derivative depends on the probability itself:
    $$h_i = \frac{\partial^2 L}{\partial z^2} = p_i (1 - p_i)$$
    *Interpretation:* The curvature is highest when $p \approx 0.5$ (uncertainty is high) and lowest when $p$ is near 0 or 1.

* **Final Output:**
    The tree outputs a "log-odds" value. To get the final probability, we apply the Sigmoid function to the sum of all tree outputs.

---

### Summary Comparison Table

| Component | **Regressor** (MSE) | **Classifier** (Log Loss) |
| :--- | :--- | :--- |
| **Loss Function ($L$)** | $\frac{1}{2}(y - \hat{y})^2$ | $-[y \ln(p) + (1-y) \ln(1-p)]$ |
| **Gradient ($g_i$)** | $\hat{y} - y$ | $p - y$ |
| **Hessian ($h_i$)** | $1$ (Constant) | $p(1-p)$ (Variable) |
| **Leaf Weight Formula** | $w^* = \frac{-\sum (\hat{y}-y)}{\sum 1 + \lambda}$ | $w^* = \frac{-\sum (p-y)}{\sum p(1-p) + \lambda}$ |

In [1]:
# Objective Function (Squared Error)
def squared_error_grad_hess(y_true,y_pred):
    # gradient: d/dy_pred (1/2 * (y - y_pred)^2) = y_pred - y
    g = y_pred - y_true
    # hessian: second derivative = 1
    h = 1.0 
    return g,h


In [2]:
# Structure of a Tree Node

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # leaf weight


Compute Leaf Weight (XGBoost Formula)
𝑤  = − 𝐺/𝐻+𝜆
​


In [3]:
def compute_leaf_weight(G, H, lambda_):
    return - G / (H + lambda_)


In [4]:
# Gain of a Split
def compute_gain(GL, HL, GR, HR, lambda_, gamma):
    parent_gain = (GL + GR)**2 / (HL + HR + lambda_)
    left_gain = GL**2 / (HL + lambda_)
    right_gain = GR**2 / (HR + lambda_)
    return 0.5 * (left_gain + right_gain - parent_gain) - gamma


In [5]:
import numpy as np


#  OBJECTIVE (Squared Error)

def squared_error_grad_hess(y_true, y_pred):
    g = y_pred - y_true  # gradient
    h = np.ones_like(y_true)  # hessian (constant)
    return g, h


#  TREE NODE

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # leaf weight


#  LEAF WEIGHT FORMULA
#  w = -G / (H + λ)

def compute_leaf_weight(G, H, lambda_):
    return -G / (H + lambda_)


#  XGBOOST GAIN FORMULA

def compute_gain(GL, HL, GR, HR, lambda_, gamma):
    parent = (GL + GR)**2 / (HL + HR + lambda_)
    left = GL**2 / (HL + lambda_)
    right = GR**2 / (HR + lambda_)
    return 0.5 * (left + right - parent) - gamma


#  FIND BEST SPLIT

def find_best_split(X, g, h, lambda_, gamma):
    best_gain = -float("inf")
    best_feature = None
    best_threshold = None

    n_samples, n_features = X.shape

    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])

        for t in thresholds:
            left_idx = X[:, feature] <= t
            right_idx = ~left_idx

            if left_idx.sum() == 0 or right_idx.sum() == 0:
                continue

            GL, HL = g[left_idx].sum(), h[left_idx].sum()
            GR, HR = g[right_idx].sum(), h[right_idx].sum()

            gain = compute_gain(GL, HL, GR, HR, lambda_, gamma)

            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = t
                
    return best_feature, best_threshold, best_gain


#  BUILD TREE (Recursive)

def build_tree(X, g, h, depth, max_depth, lambda_, gamma):
    if depth == max_depth:
        return Node(value=compute_leaf_weight(g.sum(), h.sum(), lambda_))

    feature, threshold, gain = find_best_split(X, g, h, lambda_, gamma)

    if (feature is None) or (gain < 0):
        return Node(value=compute_leaf_weight(g.sum(), h.sum(), lambda_))

    left_idx = X[:, feature] <= threshold
    right_idx = ~left_idx

    left_child = build_tree(X[left_idx], g[left_idx], h[left_idx], depth+1, max_depth, lambda_, gamma)
    right_child = build_tree(X[right_idx], g[right_idx], h[right_idx], depth+1, max_depth, lambda_, gamma)

    return Node(feature, threshold, left_child, right_child)


#  PREDICT WITH ONE TREE

def predict_tree(node, x):
    if node.value is not None:
        return node.value

    if x[node.feature] <= node.threshold:
        return predict_tree(node.left, x)
    else:
        return predict_tree(node.right, x)


#  XGBOOST REGRESSOR CORE

class XGBoostRegressorCore:
    def __init__(self, n_estimators=50, learning_rate=0.1, max_depth=3, lambda_=1, gamma=0):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.lambda_ = lambda_
        self.gamma = gamma
        self.trees = []
    
    def fit(self, X, y):
        n = len(y)
        y_pred = np.zeros(n)

        for _ in range(self.n_estimators):
            g = y_pred - y  # gradient
            h = np.ones(n)  # hessian

            tree = build_tree(X, g, h, 0, self.max_depth, self.lambda_, self.gamma)
            self.trees.append(tree)

            update = np.array([predict_tree(tree, x) for x in X])
            y_pred += self.learning_rate * update

    def predict(self, X):
        preds = np.zeros(X.shape[0])

        for tree in self.trees:
            preds += self.learning_rate * np.array([predict_tree(tree, x) for x in X])

        return preds


XGBoost is a gradient boosting algorithm:

Start with all predictions = 0

Compute gradient (how wrong the model is)

Build a decision tree that predicts the gradient

Add this tree to the model

Repeat for many trees

Each tree fixes the errors of the previous trees

In [ ]:
# If we can do this (Classifier):
from xgboost import XGBClassifier
model_clf = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    objective='binary:logistic',
    eval_metric='logloss'
)

# Then you automatically know this (Regressor):
from xgboost import XGBRegressor
model_reg = XGBRegressor(
    n_estimators=1000,           # Same
    learning_rate=0.05,          # Same  
    max_depth=6,                 # Same
    objective='reg:squarederror', # Only this changes!
    eval_metric='rmse'           # And this changes
)